In [4]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
import xml.etree.ElementTree as ET
import folium
from pyproj import Transformer
from folium import PolyLine
import re

In [6]:


# URL du fichier ZIP
zip_url = "https://parisdata.opendatasoft.com/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2020_zip/"

# Étape 1 : Télécharger le fichier ZIP depuis l'URL
response = requests.get(zip_url)
if response.status_code == 200:
    # Charger le fichier ZIP dans un buffer en mémoire
    zip_file = BytesIO(response.content)
    # Étape 2 : Ouvrir le fichier ZIP
    with zipfile.ZipFile(zip_file, 'r') as z:
        # Lister les fichiers dans le ZIP
        file_list = z.namelist()
        print(f"Fichiers disponibles dans le ZIP : {file_list}")
        
        # Filtrer les fichiers texte (si nécessaire)
        text_files = [f for f in file_list if f.endswith('.txt')]
        
        # Lire le premier fichier texte
        if text_files:
            with z.open(text_files[20]) as file:
                # Charger le fichier texte dans un DataFrame
                df = pd.read_csv(
                    file,
                    sep=';',  # Ajuster selon le séparateur utilisé
                    encoding='utf-8',  # Encodage à adapter
                    dtype=str  # Charger toutes les colonnes comme chaînes
                )
                
        else:
            print("Aucun fichier texte trouvé dans le ZIP.")
else:
    print(f"Erreur lors du téléchargement du fichier ZIP. Code HTTP : {response.status_code}")




    


Fichiers disponibles dans le ZIP : ['trafic_capteurs_2020_W00_20200101_20200108.txt', 'trafic_capteurs_2020_W01_20200108_20200115.txt', 'trafic_capteurs_2020_W02_20200115_20200122.txt', 'trafic_capteurs_2020_W03_20200122_20200129.txt', 'trafic_capteurs_2020_W04_20200129_20200201.txt', 'trafic_capteurs_2020_W04_20200201_20200208.txt', 'trafic_capteurs_2020_W05_20200208_20200215.txt', 'trafic_capteurs_2020_W06_20200215_20200222.txt', 'trafic_capteurs_2020_W07_20200222_20200229.txt', 'trafic_capteurs_2020_W08_20200229_20200301.txt', 'trafic_capteurs_2020_W08_20200301_20200308.txt', 'trafic_capteurs_2020_W09_20200308_20200315.txt', 'trafic_capteurs_2020_W10_20200315_20200322.txt', 'trafic_capteurs_2020_W11_20200322_20200329.txt', 'trafic_capteurs_2020_W12_20200329_20200401.txt', 'trafic_capteurs_2020_W13_20200401_20200408.txt', 'trafic_capteurs_2020_W14_20200408_20200415.txt', 'trafic_capteurs_2020_W15_20200415_20200422.txt', 'trafic_capteurs_2020_W16_20200422_20200429.txt', 'trafic_capteu

In [7]:
# 3. Fonction pour filtrer le dataset entre 17h et 18h
def filter(df, start_hour, end_hour):
    """
    Filtrer les données en fonction de l'heure spécifiée.
    """
    df['t_1h'] = pd.to_datetime(df['t_1h'])
    df = df[(df['t_1h'].dt.hour >= start_hour) & (df['t_1h'].dt.hour < end_hour)]

    df =df.drop_duplicates(subset=['iu_ac'])
    return df

# Filtrer les données entre 17h et 18h
df_filtered = filter(df, 17, 18)

In [8]:
df_filtered.head(2) 

,iu_ac,libelle,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,t_1h,q,k,etat_trafic,etat_barre,dessin
16,799,Bd_Kellermann,460,Bd_Kellermann-Moulin_Pointe,459,Bd_Kellermann-Damesme,2020-05-01 17:00:00,134.0,0.58056,1,3,"""<PLINE COURBE=""""1""""><PT X=""""601352"""" Y=""""1245..."
184,651,Av_Denfert_Rochereau,377,Pl_D_Roche-Bd_Raspail-Bd_Arago,378,Av_Denf_Roch-St_Vincent_de_Paul,2020-05-01 17:00:00,NaN,NaN,0,3,"""<PLINE><PT X=""""599860"""" Y=""""126220""""/><PT X=""..."


In [9]:
# 2. Fonction pour extraire les coordonnées (X, Y) de la colonne 'dessin'
def extract_coordinates(dessin):
    """
    Fonction pour extraire les coordonnées X et Y de la colonne 'dessin'.
    """
    coordinates = re.findall(r'X=""(\d+)"" Y=""(\d+)""', dessin)
    return [(int(x), int(y)) for x, y in coordinates]

# Appliquer l'extraction des coordonnées
df_filtered['coordinates'] = df_filtered['dessin'].apply(extract_coordinates)

In [10]:

# 4. Créer une carte Folium centrée sur Paris
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# 5. Ajouter les segments de route avec les nouvelles couleurs
for _, row in df_filtered.iterrows():
    coords = row['coordinates']
    traffic_state = row['etat_trafic']
    
    # Définir la couleur en fonction de l'état du trafic
    if traffic_state == 1:  # Fluide
        color = 'green'
    elif traffic_state == 2:  # Pré-saturé
        color = 'yellow'
    elif traffic_state == 3:  # Saturé
        color = 'orange'
    elif traffic_state == 4:  # Bloqué
        color = 'red'
    else:  # Inconnu
        color = 'black'
    
    # Ajouter le segment de route à la carte
    if coords:
        PolyLine(coords, color=color, weight=5).add_to(m)

# 6. Sauvegarder et afficher la carte
m
